Dataset from: https://www.kaggle.com/datasets/fanconic/skin-cancer-malignant-vs-benign/data

In [9]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torchvision.io import read_image
import os
import numpy as np

Define a class to save the dataset

In [10]:
class Dataset(Dataset):
    def __init__(self, data):
        self.x = data[:,0]
        self.y = data[:,-1]

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        y = self.y[idx]
        img_x = read_image(self.x[idx])
        return img_x/255, int(y)

Define a function that saves the path of each image with it's corresponding label inside the `data` list.

`data`:
<p>[['train/benign/233.jpg' '0']<br>
 ['train/benign/1553.jpg' '0']<br>
 ['train/benign/435.jpg' '0']<br>
 ...<br>
 ['train/malignant/1413.jpg' '1']<br>
 ['train/malignant/378.jpg' '1']<br>
 ['train/malignant/890.jpg' '1']]</p>

In [11]:
#Loads images from the determined path and labels them 1 if they
#are on the 'malignant' folder or 0 if they are ont the benign folder
def load_images_and_labels(folder_path, target_size=(224, 224)):
    data = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path):
            label = 1 if "malignant" in img_path else 0
            data.append((img_path, label))

    return np.array(data)

Save train and test data

In [12]:
#Save train and test data
train_data = np.concatenate((load_images_and_labels('dataset/train/benign'), load_images_and_labels('dataset/train/malignant')))
test_data = np.concatenate((load_images_and_labels('dataset/test/benign'), load_images_and_labels('dataset/test/malignant')))

#Save train and test data in Dataset objects
train_dataset = Dataset(train_data)
test_dataset = Dataset(test_data)

#Create Dataloaders (Dataloader iterates over the dataset batch by batch)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

Define wich device the model will train on.

In [13]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


Define the neural network

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.softmax = nn.Softmax()
        self.internal_nn_stack = nn.Sequential(        
            nn.Conv2d(3, 96, (11,11), stride=4), 
            nn.MaxPool2d((3,3)),
            nn.Conv2d(96, 256, (3,3), padding=2), 
            nn.MaxPool2d((3,3), stride= 2), 
            nn.Conv2d(256, 256, (2,2), padding=2),
            nn.MaxPool2d((3,3), stride= 2),
            nn.Flatten(), 

            nn.Linear(6400, 3000), 
            nn.ReLU(),
            nn.Linear(3000, 300), 
            nn.ReLU(),
            nn.Linear(300, 30),
            nn.ReLU(),
            nn.Linear(30, 2)
        )

    def forward(self, x):
        logits = self.internal_nn_stack(x)
        return logits

Define optimization loops

In [15]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
#Instantiate the model
model = NeuralNetwork().to(device)

#Define the hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 10

#Define what loss function will be used
loss_fn = nn.CrossEntropyLoss()

#Define what optimization algorithm will be used
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 0.692277  [   64/ 2637]
Test Error: 
 Accuracy: 68.6%, Avg loss: 0.677687 

Epoch 2
-------------------------------
loss: 0.676856  [   64/ 2637]
Test Error: 
 Accuracy: 45.5%, Avg loss: 0.767749 

Epoch 3
-------------------------------
loss: 0.788127  [   64/ 2637]
Test Error: 
 Accuracy: 47.1%, Avg loss: 0.697360 

Epoch 4
-------------------------------
loss: 0.697901  [   64/ 2637]
Test Error: 
 Accuracy: 69.1%, Avg loss: 0.603728 

Epoch 5
-------------------------------
loss: 0.571380  [   64/ 2637]
Test Error: 
 Accuracy: 73.9%, Avg loss: 0.503163 

Epoch 6
-------------------------------
loss: 0.629552  [   64/ 2637]
Test Error: 
 Accuracy: 68.0%, Avg loss: 0.531176 

Epoch 7
-------------------------------
loss: 0.519648  [   64/ 2637]
Test Error: 
 Accuracy: 55.9%, Avg loss: 0.852592 

Epoch 8
-------------------------------
loss: 0.717821  [   64/ 2637]
Test Error: 
 Accuracy: 79.7%, Avg loss: 0.392050 

Epoch 9
----------------

Save model

In [17]:
torch.save(model, 'model.pth')